In [ ]:
import pandas as pd 
import numpy as np 
from tensorflow.keras.layers import Flatten,Input,Dense,GlobalAveragePooling2D
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.applications.vgg19 import VGG19,preprocess_input
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing import image
import os
import cv2
import matplotlib.pyplot as plt
import random

# Functions

In [ ]:
def give_dataset(m):
    data = []
    if m=='train':
            path = os.path.join('dataset','training_set')
    elif m=='test':
            path = os.path.join('dataset','test_set')
    else:
        print('String should be train ot test')
    for f in os.listdir(path):
        for im in os.listdir(os.path.join(path,f)):
            image = cv2.imread(os.path.join(path,f,im),cv2.IMREAD_COLOR)
            image = cv2.resize(image,(224,224))
            if im[:3]=='cat':
                data.append([image , 0])
            if im[:3]=='dog':
                data.append([image ,1])
    random.shuffle(data)
    return data


In [ ]:
def give_splits(dataset):
    x= []
    y=[]
    for d,t in dataset:
        x.append(d)
        y.append(t)
    x= np.array(x).reshape(-1,224,224,3)
    y = np.array(y)
    return x,y

In [ ]:
train_data = give_dataset('train')

In [ ]:
test_data = give_dataset('test')

In [ ]:
x_train,y_train = give_splits(train_data)

In [ ]:
x_test,y_test = give_splits(test_data)

In [ ]:
np.save('dataset/x_train.npy',x_train)
np.save('dataset/y_train.npy',y_train)
np.save('dataset/x_test.npy',x_test)
np.save('dataset/y_test.npy',y_test)

In [ ]:
x_train = np.load(r'D:\Data Analytics for Business\Keras learning\dataset\x_train.npy',)
y_train = np.load(r'D:\Data Analytics for Business\Keras learning\dataset\y_train.npy')
x_test = np.load(r'D:\Data Analytics for Business\Keras learning\dataset\x_test.npy')
y_test = np.load(r'D:\Data Analytics for Business\Keras learning\dataset\y_test.npy')

In [ ]:
vgg = VGG19(weights='imagenet',include_top=False)
for layer in vgg.layers:
    layer.trainable=False

In [ ]:
def give_model(optimizer='adam'):
    x= vgg.output
    x= GlobalAveragePooling2D()(x)
    x = Dense(x_train.shape[0],activation='relu')(x)
    out = Dense(1,activation='sigmoid')(x)
    model = Model(inputs=vgg.input,outputs=out)
    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return model

# Model

In [ ]:
new_model = Model(inputs= model.input,outputs=out)

In [ ]:
new_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
hist = new_model.fit(x_train,y_train)

In [ ]:
hist.history

# USing Randomized search

In [ ]:
knn = KerasClassifier(build_fn=give_model)

In [ ]:
params = {'optimizer':['adam','sgd'],'batch_size':[16,32,64]}

In [ ]:
cv = RandomizedSearchCV(knn,params)

In [ ]:
cv.fit(x_train,y_train)

In [ ]:
cv.best_params_

In [ ]:
use = give_model(optimizer='sgd')

In [ ]:
chkpt = ModelCheckpoint('dataset/cat-dog.hdf5',monitor='val_loss',verbose=0)

In [ ]:
hist = use.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[chkpt],batch_size=16,epochs=12)

In [ ]:
plt.plot(hist.history['val_accuracy'])
plt.plot(hist.history['accuracy'])

In [ ]:
plt.hist.history['val_accuracy']

In [ ]:
def give_model_new(optimizer='adam',vgg=vgg):
    x= vgg.output
    x= GlobalAveragePooling2D()(x)
    x = Dense(8000,activation='relu')(x)
    out = Dense(1,activation='sigmoid')(x)
    model = Model(inputs=vgg.input,outputs=out)
    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return model

def give_result():
    # path = os.path.join('dataset\static',filename)
    im = image.load_img(r'D:\Data Analytics for Business\Keras learning\dog vs cat\dataset\training_set\cats\cat.1.jpg')
    im = image.smart_resize(im,(224,224))
    im = im.reshape(-1,224,224,3)
    model = give_model()
    model.load_weights('D:\Data Analytics for Business\Keras learning\dataset\dataset\cat-dog.hdf5')
    print(model.predict(im))

In [ ]:
mod = give_model_new()
mod.summary()

In [ ]:
mod.load_weights('D:\Data Analytics for Business\Keras learning\dataset\dataset\cat-dog.hdf5')
mod.summary()

In [ ]:
np.argmax(mod.predict(im))

In [ ]:
im = cv2.imread(r'D:\Data Analytics for Business\Keras learning\dataset\training_set\dogs\dog.23.jpg')
im = cv2.resize(im,(224,224))
im = im.reshape(-1,224,224,3)
# im = image.img_to_array(im)

In [ ]:
tim = im.reshape(-1,224,224,3)
tim.shape